In [16]:
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os

In [2]:
Id = []
import numpy as np
#data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(r"E:\LiveProject\Deep_Learning\farm_insects\data"):
    for filename in filenames:
        Id.append(os.path.join(dirname, filename))
Id[:5]

['E:\\LiveProject\\Deep_Learning\\farm_insects\\data\\Africanized Honey Bees (Killer Bees)\\Image_1.jpg',
 'E:\\LiveProject\\Deep_Learning\\farm_insects\\data\\Africanized Honey Bees (Killer Bees)\\Image_10.jpg',
 'E:\\LiveProject\\Deep_Learning\\farm_insects\\data\\Africanized Honey Bees (Killer Bees)\\Image_100.jpg',
 'E:\\LiveProject\\Deep_Learning\\farm_insects\\data\\Africanized Honey Bees (Killer Bees)\\Image_102.jpg',
 'E:\\LiveProject\\Deep_Learning\\farm_insects\\data\\Africanized Honey Bees (Killer Bees)\\Image_104.jpg']

In [3]:
df=pd.DataFrame()
df=df.assign(filename=Id)
df.head()

,filename
0,E:\LiveProject\Deep_Learning\farm_insects\data...
1,E:\LiveProject\Deep_Learning\farm_insects\data...
2,E:\LiveProject\Deep_Learning\farm_insects\data...
3,E:\LiveProject\Deep_Learning\farm_insects\data...
4,E:\LiveProject\Deep_Learning\farm_insects\data...


In [4]:
df. shape

(1591, 1)

In [5]:
df['label']=df['filename']
df['label']=df['label']. str. replace(r"E:\\LiveProject\\Deep_Learning\\farm_insects\\data", '')
df.head()

C:\Users\debna\AppData\Local\Temp\ipykernel_21660\1560752447.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['label']=df['label']. str. replace(r"E:\\LiveProject\\Deep_Learning\\farm_insects\\data", '')


,filename,label
0,E:\LiveProject\Deep_Learning\farm_insects\data...,\Africanized Honey Bees (Killer Bees)\Image_1.jpg
1,E:\LiveProject\Deep_Learning\farm_insects\data...,\Africanized Honey Bees (Killer Bees)\Image_10...
2,E:\LiveProject\Deep_Learning\farm_insects\data...,\Africanized Honey Bees (Killer Bees)\Image_10...
3,E:\LiveProject\Deep_Learning\farm_insects\data...,\Africanized Honey Bees (Killer Bees)\Image_10...
4,E:\LiveProject\Deep_Learning\farm_insects\data...,\Africanized Honey Bees (Killer Bees)\Image_10...


In [6]:
# Removing the unwanted values like ?/
import re
pattern = r"\\([^\\]+)"
df['label'] = df['label'].str.extract(pattern, expand=False)

In [7]:
df. label. value_counts()

Spider Mites                            119
Corn Borers                             115
Brown Marmorated Stink Bugs             114
Fall Armyworms                          113
Colorado Potato Beetles                 112
Corn Earworms                           110
Thrips                                  109
Tomato Hornworms                        109
Cabbage Loopers                         104
Citrus Canker                           104
Fruit Flies                             101
Western Corn Rootworms                  100
Africanized Honey Bees (Killer Bees)     97
Armyworms                                96
Aphids                                   88
Name: label, dtype: int64

In [8]:
# Replace the label
df['label'] = df['label'].replace('Africanized Honey Bees (Killer Bees)', 'Africanized Honey Bees Killer Bees')

In [9]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

In [10]:
#Define the image data generators for training and testing
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
# Generate augmented images for training
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    x_col="filename",
                                                    y_col="label",
                                                    target_size=(100, 100),
                                                    batch_size=32,
                                                    class_mode="categorical")

C:\Users\debna\anaconda3\lib\site-packages\keras\preprocessing\image.py:1137: UserWarning: Found 6 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 1107 validated image filenames belonging to 15 classes.


In [12]:
# Generate augmented images for testing
test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                                  x_col="filename",
                                                  y_col="label",
                                                  target_size=(100, 100),
                                                  batch_size=32,
                                                  class_mode="categorical")

Found 472 validated image filenames belonging to 15 classes.


C:\Users\debna\anaconda3\lib\site-packages\keras\preprocessing\image.py:1137: UserWarning: Found 6 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [20]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam

In [14]:
# Create the MobileNet base model
base_model = MobileNet(include_top=False, input_shape=(100, 100, 3))

17225924/17225924 [==============================] - 5s 0us/step


In [17]:
# Build the model architecture
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(15, activation='softmax'))

In [18]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [21]:
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
# Train the model
history = model.fit(train_generator, epochs=20, validation_data=test_generator)

C:\Users\debna\anaconda3\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/20
35/35 [==============================] - 40s 1s/step - loss: 3.4939 - accuracy: 0.1427 - val_loss: 2.5313 - val_accuracy: 0.3093
Epoch 2/20
35/35 [==============================] - 33s 949ms/step - loss: 2.0524 - accuracy: 0.3776 - val_loss: 2.1307 - val_accuracy: 0.4025
Epoch 3/20
35/35 [==============================] - 33s 958ms/step - loss: 1.5807 - accuracy: 0.5023 - val_loss: 2.0321 - val_accuracy: 0.4174
Epoch 4/20
35/35 [==============================] - 33s 946ms/step - loss: 1.3570 - accuracy: 0.5682 - val_loss: 1.8790 - val_accuracy: 0.4513
Epoch 5/20
35/35 [==============================] - 33s 951ms/step - loss: 1.1871 - accuracy: 0.6188 - val_loss: 1.8431 - val_accuracy: 0.4576
Epoch 6/20
35/35 [==============================] - 33s 957ms/step - loss: 1.0086 - accuracy: 0.6838 - val_loss: 1.8148 - val_accuracy: 0.4767
Epoch 7/20
35/35 [==============================] - 33s 955ms/step - loss: 0.9250 - accuracy: 0.7164 - val_loss: 1.7523 - val_accuracy: 0.5148
Ep

In [23]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)

15/15 [==============================] - 9s 613ms/step - loss: 1.8014 - accuracy: 0.5233


In [24]:
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Test Loss: 1.8014053106307983
Test Accuracy: 0.5233050584793091


In [25]:
# save it as a h5 file
from tensorflow.keras.models import load_model
model.save('pest_mobilenet.h3')

INFO:tensorflow:Assets written to: pest_mobilenet.h3\assets


INFO:tensorflow:Assets written to: pest_mobilenet.h3\assets


# MobileNetV2:

In [27]:
from tensorflow.keras.applications import MobileNetV2

In [28]:
# Create the MobileNet base model
base_model_mobilenetv2 = MobileNetV2(include_top=False, input_shape=(224, 224, 3))

9406464/9406464 [==============================] - 1s 0us/step


In [29]:
# Build the model architecture
model_mobilenetv2 = Sequential()
model_mobilenetv2.add(base_model)
model_mobilenetv2.add(GlobalAveragePooling2D())
model_mobilenetv2.add(Dense(15, activation='softmax'))

In [30]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model_mobilenetv2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
# Train the model
history_v2 = model_mobilenetv2.fit(train_generator, epochs=25, validation_data=test_generator)

Epoch 1/25
35/35 [==============================] - 35s 970ms/step - loss: 3.4400 - accuracy: 0.1427 - val_loss: 2.5839 - val_accuracy: 0.2458
Epoch 2/25
35/35 [==============================] - 33s 954ms/step - loss: 2.0268 - accuracy: 0.3758 - val_loss: 2.0493 - val_accuracy: 0.3898
Epoch 3/25
35/35 [==============================] - 35s 1s/step - loss: 1.5951 - accuracy: 0.5086 - val_loss: 1.9472 - val_accuracy: 0.4237
Epoch 4/25
35/35 [==============================] - 33s 950ms/step - loss: 1.3297 - accuracy: 0.5899 - val_loss: 1.8302 - val_accuracy: 0.4407
Epoch 5/25
35/35 [==============================] - 36s 1s/step - loss: 1.1304 - accuracy: 0.6450 - val_loss: 1.7744 - val_accuracy: 0.4767
Epoch 6/25
35/35 [==============================] - 33s 939ms/step - loss: 1.0315 - accuracy: 0.6694 - val_loss: 1.7261 - val_accuracy: 0.4915
Epoch 7/25
35/35 [==============================] - 33s 948ms/step - loss: 0.9003 - accuracy: 0.7218 - val_loss: 1.7178 - val_accuracy: 0.5148
Epoch

In [33]:
# Evaluate the model on the test set
test_loss_v2, test_accuracy_v2 = model.evaluate(test_generator)

15/15 [==============================] - 9s 597ms/step - loss: 1.9780 - accuracy: 0.5466


In [34]:
print(f"Test Loss: {test_loss_v2}")
print(f"Test Accuracy: {test_accuracy_v2}")

Test Loss: 1.9780179262161255
Test Accuracy: 0.5466101765632629


In [ ]:
from tensorflow.keras.applications import MobileNetV3Large
# Rest of the code is similar to MobileNet